In [1]:
# save just the green channel, each MROI strip as an h5 file
import os
import suite2p
from suite2p.run_s2p import run_s2p
import json
import sys
import numpy as np
import time
import gc
from pathlib import Path
from glob import glob
from ScanImageTiffReader import ScanImageTiffReader
import h5py
from scipy.io import savemat
import multiprocessing
import logging


def tiffs2array_nocrop(movie_list, planes, channels):
    data = [LoadTif(str(mov),planes,channels) for mov in movie_list] #CAREFUL HERE should slice after concatenation for Satsuma RF, but NOT for Frankenrig
    data = np.concatenate(data)
    print('concatenated data from all tiffs, total size: ' + str(data.shape))
    return data

def LoadTif(mov_path,planes,channels):
    with ScanImageTiffReader(mov_path) as reader:
        data = reader.data()
    Tend = int( np.floor(data.shape[0]/(planes*channels)) * (planes*channels) )
    t_slice = slice(0,Tend, 1)
    data = data[t_slice,:,:]
    return data

def save_h5_fullmeso(movs,tiff_folder,out_path,expno, planes, channels, opsjson):
    # takes a list of tifs. Concatenates each plane, then slices and crops the big movie, then saves it as .h5 file in a folder inside tiff_folder\
    # process multiple folders
    # instead of making one h5 per plane, just make it into one big h5
    # movs: list of tifs to be processed
    # tif folder: directory where the result h5 file will be saved (inside the corresponding plane folder)
    # x_slice: cropping of the FOV in x
    # x_slice: cropping of the FOV in y
    # planes: number of planes for each tif file
    # channels: number of channels"""
    tic = time.time()

    strips = np.shape(opsjson['lines'])[0]
    #exp_name1 = exp_name.replace("//","")
    outname = [0]*strips
    outdir=[0]*strips
    try:
        os.mkdir(out_path)
    except:
        if os.path.isdir(out_path):
            print('directory already exists')
        else:
            print('Cannot create directory.. probably the name is wrong')
            print(out_path)
            

    fullmov = tiffs2array_nocrop(movs,planes,channels)
    print(fullmov.shape)
    
    ylen = np.shape(opsjson['lines'])[1]
    xlen = np.shape(opsjson['lines'])[0]*opsjson['dx'][1]
    xbounds = opsjson['dx'].copy()
    xbounds.append(xlen)
    for istrip in range(strips):
        stripmov = fullmov[:,opsjson['lines'][istrip],:].copy()
        
        t_slice = slice(0,stripmov.shape[0], channels)
        x_slice = slice(0,stripmov.shape[2], 1)
        y_slice = slice(0,stripmov.shape[1], 1)
        # x_slice = slice(0,None, 1) #x_slice = slice(0,cropped_mov.shape[2], 1)
        # y_slice = slice(0, None, 1)#y_slice = slice(0,cropped_mov.shape[1], 1)
        outdir[istrip] = out_path +'plane_'+ str(istrip) + '//'
        try:
            os.mkdir(outdir[istrip])
        except:
            print('couldnt make directory')
            print(str(outdir[istrip]))

        outname[istrip]=outdir[istrip] + 'strip_mov_' + str(expno) + '.h5'
        print(f'now processing {outname[istrip]}')
        # try:
        hf = h5py.File(outname[istrip], 'a')
        stripmov1 = stripmov[t_slice, y_slice, x_slice]
        print(stripmov1.shape)
        hf.create_dataset('data', data=stripmov1, dtype= 'uint16')
        hf.close()
        print('done saving ' + str(outname[istrip]))
        # delete the array and clean RAM
        del stripmov1
        gc.collect()
        # except:
        #     print(f'the file {outname[istrip]} already exists')

    toc = time.time() - tic
    print('All saved,took  ' + str(toc) + 'secs')

    return


In [2]:
tic1 = time.time()

# INFO ABOUT YOUR FILES
baseFreq = 3.09
channels = 1
planes = 1

# drive = 'M://'
# user_name = 'ICmesoholoexpts_scanimage'
drive = 'D://'
user_name = 'HS'
mouse = 'HS_Ai203_2'
date = '220531'
expList = ['retinotopy2', 'staticICtxi0', 'staticgratiings']
#where your tifs are
tiffFolderList = [drive+ user_name + '//' + mouse +'//' + date + '//' + exp_name + '//' for exp_name in expList]
#where you want to save the output hdf5 file
suffix = 'CLFullFOV//' #'suite2pAnalysis//'
data_path = drive+ user_name + '//' + mouse +'//' + date + '//'
out_path = drive+ user_name + '//' + mouse +'//' + date + '//' + suffix

In [3]:
# only took 170 sec
tic1 = time.time()

os.chdir(data_path)
with open('ops.json') as f:
    opsjson = json.load(f)
print(opsjson.keys())

gc.collect()
for exp_name, tiff_folder in zip(expList,tiffFolderList):
    gc.collect()
    pth = Path(tiff_folder)
    movs = list(pth.glob('*.tif'))
    save_h5_fullmeso(movs,tiff_folder,out_path,exp_name, planes, channels, opsjson)

toc1 = time.time()-tic1
print(toc1)
print('completed h5 conversion')


dict_keys(['fs', 'nplanes', 'nrois', 'mesoscan', 'diameter', 'num_workers_roi', 'keep_movie_raw', 'delete_bin', 'batch_size', 'nimg_init', 'tau', 'combined', 'nonrigid', 'dx', 'dy', 'lines', 'data_path', 'save_path0'])
concatenated data from all tiffs, total size: (1096, 7620, 608)
(1096, 7620, 608)
now processing D://HS//HS_Ai203_2//220531//CLFullFOV//plane_0//strip_mov_retinotopy2.h5
(1096, 1484, 608)
done saving D://HS//HS_Ai203_2//220531//CLFullFOV//plane_0//strip_mov_retinotopy2.h5
now processing D://HS//HS_Ai203_2//220531//CLFullFOV//plane_1//strip_mov_retinotopy2.h5
(1096, 1484, 608)
done saving D://HS//HS_Ai203_2//220531//CLFullFOV//plane_1//strip_mov_retinotopy2.h5
now processing D://HS//HS_Ai203_2//220531//CLFullFOV//plane_2//strip_mov_retinotopy2.h5
(1096, 1484, 608)
done saving D://HS//HS_Ai203_2//220531//CLFullFOV//plane_2//strip_mov_retinotopy2.h5
now processing D://HS//HS_Ai203_2//220531//CLFullFOV//plane_3//strip_mov_retinotopy2.h5
(1096, 1484, 608)
done saving D://HS//

In [4]:
# two settings that are most critical for speed in low SNR recordings:
# 'max_iterations' and 'threshold_scaling'
ops = {'look_one_level_down': 0.0,
 'fast_disk': [],
 'delete_bin': True,
 'mesoscan': False,
 'bruker': False,
 'h5py': [],
 'h5py_key': 'data',
 'save_path0': [],
 'save_folder': [],
 'subfolders': [],
 'move_bin': False,
 'nplanes': 1,
 'nchannels': 1,
 'functional_chan': 1,
 'tau': 1.5,
 'fs': baseFreq/planes,
 'force_sktiff': False,
 'frames_include': -1,
 'multiplane_parallel': False,
 'preclassify': 0.0,
 'save_mat': True,
 'save_NWB': False,
 'combined': 1.0,
 'aspect': 2.0,
 'do_bidiphase': True,
 'bidiphase': 0,
#  'bidi_corrected': False,
 'do_registration': 1,
 'two_step_registration': False,
 'keep_movie_raw': False, # if two_step_registration = 1 this has to also be true.
 'nimg_init': 1000,
 'batch_size': 2000,
 'maxregshift': 0.1,
 'align_by_chan': 1,
 'reg_tif': False,
 'reg_tif_chan2': False,
 'subpixel': 10,
 'smooth_sigma_time': 1.0, # default 0
 'smooth_sigma': 1.15, # pixels, default 1.15
 'th_badframes': 1.0,
 'pad_fft': False,
 'nonrigid': False,
 'block_size': [128, 128],
 'snr_thresh': 1.2,
 'maxregshiftNR': 5.0,
 '1Preg': False,
 'spatial_hp': 25,
 'spatial_hp_reg': 26.0,
 'spatial_hp_detect': 25,
 'pre_smooth': 0,
 'spatial_taper': 50.0,
 'roidetect': True,
 'spikedetect': True,
 'sparse_mode': False,
 'diameter': 9,
 'spatial_scale': 1, # default 0, if set to 0, then the algorithm determines it automatically (recommend this on the first try)
 'connected': True,
 'nbinned': 5000, # maximum number of binned frames to use for ROI detection
 'max_iterations': 20, # default 20. how many iterations over which to extract cells - at most ops[‘max_iterations’], 
                        #but usually stops before due to ops[‘threshold_scaling’] criterion
 'threshold_scaling': 2.0, # default: 5.0. this controls the threshold at which to detect ROIs.. if you set this higher, then fewer ROIs will be detected
 'max_overlap': 0.5, # default: 0.75
 'high_pass': 100, #  default: 100. time window size
 'inner_neuropil_radius': 2, # default: 2. number of pixels to keep between ROI and neuropil donut
 'min_neuropil_pixels': 350, #default: 350. minimum number of pixels used to compute neuropil for each cell
 'allow_overlap': False, #default: False. whether or not to extract signals from pixels which belong to two ROIs.
 'chan2_thres': 0.65,
 'baseline': 'maximin',
 'win_baseline': 60.0,
 'sig_baseline': 10.0,
 'prctile_baseline': 8.0,
 'neucoeff': 0.7}

In [5]:
#list of files to analyze:
strips = np.shape(opsjson['lines'])[0]
outname = [0]*strips
outdir=[0]*strips 

#save output data
for i in range(strips):
    outdir[i]= out_path + 'plane_'+str(i)+'//'
    outname[i]= out_path + 'plane_'+str(i)+'//'

print(outdir)
print(outname[0])

['D://HS//HS_Ai203_2//220531//CLFullFOV//plane_0//', 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_1//', 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_2//', 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_3//', 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_4//']
D://HS//HS_Ai203_2//220531//CLFullFOV//plane_0//


In [6]:
# prepare the processes for each plane
db = []
jobs = []
if __name__ == '__main__':
    
    for i in range(strips):
    #while the # of planes is less than the max number of cores we want to use (leaving 2 cores for scanimage)
       if i<os.cpu_count()-2:
    # Define the dataset
            this_db = {
              'h5py':outname[i], # a single h5 file path[p]
              'h5py_key': ['data'],
              'look_one_level_down': True, # whether to look in ALL subfolders when searching for tiffs,
              'save_path0': outdir[i],
              'data_path':  [], # a list of folders with tiffs
                                                     # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
              'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
              'fast_disk': [] }# string which specifies where the binary file will be stored (should be an SSD)      
            print(f'Will be processing this data : {this_db}')
            db.append(this_db)
            p = multiprocessing.Process(target=run_s2p, args=(ops,this_db))
            p.start()          
            jobs.append(p)
       else:
            print(f'Hey, do you really want to run all these {i} cores at the same time?')

Will be processing this data : {'h5py': 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_0//', 'h5py_key': ['data'], 'look_one_level_down': True, 'save_path0': 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_0//', 'data_path': [], 'subfolders': [], 'fast_disk': []}
Will be processing this data : {'h5py': 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_1//', 'h5py_key': ['data'], 'look_one_level_down': True, 'save_path0': 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_1//', 'data_path': [], 'subfolders': [], 'fast_disk': []}
Will be processing this data : {'h5py': 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_2//', 'h5py_key': ['data'], 'look_one_level_down': True, 'save_path0': 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_2//', 'data_path': [], 'subfolders': [], 'fast_disk': []}
Will be processing this data : {'h5py': 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_3//', 'h5py_key': ['data'], 'look_one_level_down': True, 'save_path0': 'D://HS//HS_Ai203_2//220531//CLFullFOV//plane_3//', 'data_pa

In [7]:
# suite2p took 2244.20 sec, or 38 min
logging.basicConfig(level=logging.INFO)
logging.info('Starting suite2p parallel processing in a different CPU core for each MROI strips')

#run each strips in parallel
tic = time.time()
if len(jobs)<os.cpu_count()-2:
    for job in jobs:
        #print ('Parallel processing:  ' + str(outname[i]))
        job.join()
    toc = time.time() - tic
    print('All saved,took  ' + str(toc) + 'secs')
else:
   print(f'Hey, do you really want to run all these {i} cores at the same time?')

toc2 = time.time() - tic1
print('Preprocessing took ' + str(toc2/60) + 'mins')

INFO:root:Starting suite2p parallel processing in a different CPU core for each MROI strips


NameError: name 'tic' is not defined